In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

<h1> PREPROCESSING </h1>

In [2]:
df = pd.read_csv('/content/Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
x = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [4]:
categorical_columns = [col for col in x.columns if x[col].dtype == 'object']
numerical_columns = [col for col in x.columns if x[col].dtype != 'object']

In [5]:
transformer = ColumnTransformer([
    ('cat', OneHotEncoder(sparse_output=False, drop='first'), categorical_columns),
    ('num', 'passthrough', numerical_columns)
])

<h1>TRAIN TEST SPLIT </h1>

In [6]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state=42)

In [7]:
x_train  = transformer.fit_transform(x_train)
x_test = transformer.transform(x_test)

In [8]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

# Standardize target
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y.values.reshape(-1, 1))
y_test = sc_y.transform(y.values.reshape(-1, 1))

In [26]:
class customdataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]



In [27]:
train_dataset = customdataset(x_train, y_train)
test_dataset = customdataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

<h1>MODEL ARCHITECTURE<h1>

In [28]:
class myANN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)



In [29]:
model = myANN(x_train.shape[1])
epochs = 100
lr = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

<h1>MODEL TRAINING<h1>

In [30]:
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/100, Loss: 11.9291
Epoch 10/100, Loss: 4.1622
Epoch 20/100, Loss: 3.4212
Epoch 30/100, Loss: 3.1265
Epoch 40/100, Loss: 2.8767
Epoch 50/100, Loss: 2.5382
Epoch 60/100, Loss: 2.3371
Epoch 70/100, Loss: 2.4452
Epoch 80/100, Loss: 2.0711
Epoch 90/100, Loss: 2.0115
Epoch 100/100, Loss: 2.0086


In [31]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())


In [32]:
y_true = sc_y.inverse_transform(np.array(y_true).reshape(-1, 1)).flatten()
y_pred = sc_y.inverse_transform(np.array(y_pred).reshape(-1, 1)).flatten()

In [33]:

print(f"MSE: {mean_squared_error(y_true, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_true, y_pred):.4f}")
print(f"R² Score: {r2_score(y_true, y_pred):.4f}")

MSE: 1896327151616.0000
MAE: 1008551.8750
R² Score: 0.6248
